In [8]:
import pandas as pd
import os,re
import numpy as np

In [9]:
parsed = os.listdir('Data')
parsed = [i for i in parsed if '2022' in i]

In [10]:
codes = {
    2011:{'MT':130,'CN':123,'CH':119,'LC':126},
    2012:{'CN':153,'CH':154,'LC':155,'MT':156},
    2013:{'CN':188,'CH':187,'LC':189,'MT':190},
    2015:{'CN':277,'CH':273,'LC':280,'MT':284},
    2016:{'CN':307,'CH':308,'LC':309,'MT':310},
    2017:{'CN':407,'CH':408,'LC':409,'MT':410},
    2018:{'CN':463,'CH':464,'LC':465,'MT':466},
    2019:{'CN':519,'CH':520,'LC':521,'MT':522},
    2020:{'CN':604,'CH':574,'LC':584,'MT':594},
    2021:{'CN':916,'CH':886,'LC':896,'MT':906},
    2022:{'CN':1092,'CH':1062,'LC':1072,'MT':1082},
    2023:{'CN':0,'CH':0,'LC':0,'MT':0}
}

In [14]:
dfs = {}


def match_gab_to_df(df,gab):
    answers=[]
    languages=[]
    used = set()
    for line in df.to_dict(orient='records'):
        if line['test']=='redação':
            answers.append('')
            languages.append('portuguese')
            continue
        possible_gabarito = gab[gab['question']==line['question_number']]
        gabarito = possible_gabarito.iloc[0]
        
        if (str(gabarito['question'])+str(gabarito['test'])) in used:
            gabarito = possible_gabarito.iloc[1]
            
        answers.append(gabarito['answer'])
        languages.append(gabarito['test'] if not pd.isna(gabarito['test']) else 'portuguese')
        used.add(str(gabarito['question'])+str(gabarito['test']))
    return answers,languages


for prova in parsed:
    print(prova)
    df = pd.read_csv('Data/'+prova)
    print(len(df))
    year = int(re.search("\d\d\d\d",prova).group())
    df['question'] = df['question'].apply(lambda question: question.upper())
    df['year'] = year
    df['question_number'] = df['question'].apply(lambda question: int(re.search("[\d]+",question).group()) if re.search("[\d]+",question) else np.nan)
    df['test']='redação'
    if 'D1_' in prova or 'ia1' in prova or 'DIA_1' in prova:
        if year < 2017:
            df.loc[df['question_number']<=45,'test'] = 'CH'
            df.loc[df['question_number']>45,'test'] = 'CN'
        else:
            df.loc[df['question_number']<=45,'test'] = 'LC'
            df.loc[df['question_number']>45,'test'] = 'CH'
    else:
        if year < 2017:
            df.loc[df['question_number']<=135,'test'] = 'LC'
            df.loc[df['question_number']>135,'test'] = 'MT'
        else:
            df.loc[df['question_number']<=135,'test'] = 'CN'
            df.loc[df['question_number']>135,'test'] = 'MT'
    #add gabarito data
    gab = pd.read_csv('GabaritosCsv/'+prova.replace('CAD','GAB')).sort_values(by=['question','test'])
    df['answer'],df['language'] = match_gab_to_df(df,gab)
    #we want to sort dataframe in the following order: english -> spanish -> portuguese then question orders
    #this is useful for the adding of microdata
    df['sort_index'] = df['language'].apply(lambda lang: 0 if lang == 'english' else 1 if lang=='spanish' else 2)
    df = df.sort_values(by=['sort_index','question_number']).drop(columns=['sort_index'])
    
    #add microdata data
    important_columns = ['CO_PROVA','NU_PARAM_A','NU_PARAM_B','NU_PARAM_C','TP_LINGUA','CO_POSICAO','CO_HABILIDADE','TX_GABARITO']
    microdata = pd.read_csv(f'microdados/ITENS_PROVA_{year}.csv',sep=';',usecols=important_columns, encoding='latin-1')
    microdata.loc[microdata['TP_LINGUA'].isna(),'TP_LINGUA']='portuguese'
    microdata.loc[microdata['TP_LINGUA']==0,'TP_LINGUA']='english'
    microdata.loc[microdata['TP_LINGUA']==1,'TP_LINGUA']='spanish'
    
    #fix maths in the answers
    for col in ['A','B','C','D','E','body']:
        df[col] = df[col].apply(lambda text: re.sub(' ?,? ?[Ff]echa parêntese ?,?',')',re.sub(' ?,? ?[Aa]bre parêntese ?,? ?',' (',text)) if type(text)==str else text)
    
#     #divide tests
    for test in df['test'].drop_duplicates().tolist():
#         print(prova+' '+ test)
        one_subject_test = df[df['test']==test].copy().reset_index(drop=True)
        if test == 'redação':
            one_subject_test.to_csv(f'ProvasComMicrodados/ENEM_{year}_{test}.csv')
            continue
        else:
            CO_PROVA = codes[year][test]
#             specific_microdata = microdata[microdata['CO_PROVA']==CO_PROVA].copy()
#             one_subject_test['CO_PROVA'] = CO_PROVA

#             try:
#                 output = pd.merge(one_subject_test,specific_microdata,left_on=['question_number','language'],right_on=['CO_POSICAO','TP_LINGUA'])
#                 if output.empty:
#                     raise Exception(f'Failed matching')
#             except:
#                 one_subject_test['match'] = list(one_subject_test.index+1)
#                 output = pd.merge(one_subject_test,specific_microdata,left_on=['match'],right_on=['CO_POSICAO']).drop(columns=['match'])
#                 if not (output['TX_GABARITO']!=output['answer']).sum()==0:
#                     print(f'Gabarito and answer should be the same for test {prova} {test}')
#                     #CO_PROVA = str(CO_PROVA) + 'MAYBEBROKENDONOTUSE'
#             if output.empty:
#                 print('Output empty??')
#             if len(output)!=45 and len(output)!=50:
#                 raise Exception('Inconsistent Output')
#             output.drop(columns=['CO_PROVA_x','TP_LINGUA','CO_POSICAO'],inplace=True)
#             output.rename(columns={'CO_PROVA_y':'CO_PROVA'},inplace=True)
#             output['question_number'] = output['question_number'].astype(int)
        one_subject_test['question_number']=one_subject_test['question_number'].astype(int)
        if len(one_subject_test)!=45 and len(one_subject_test)!=50:
            print(prova,len(one_subject_test),test)
        one_subject_test.to_csv(f'ProvasComMicrodados/ENEM_{year}_{test}_CO_PROVA_{CO_PROVA}.csv',index=False,encoding='utf-8')
    dfs[prova] = df

ENEM_2022_P1_CAD_09_DIA_1_LARANJA_LEDOR.csv
96
ENEM_2022_P1_CAD_11_DIA_2_LARANJA_LEDOR.csv
90


In [ ]:
#remove dumb parsing artifacts from specific provas
year=2022
base_path = "./ProvasComMicrodados/"
enems = os.listdir(base_path)
df = pd.concat([pd.read_csv(base_path+file) for file in enems ]).sort_values('question_number').reset_index(drop=True)
for file in enems:
    if '2022' in file and 'redação' not in file:
        df = pd.read_csv(base_path+file)
        df['A'] = df['A'].str.lstrip('A').str.strip(' ')
        print(df['A'].isna().sum())
        df['B'] = df['B'].str.lstrip('B').str.strip(' ')
        print(df['B'].isna().sum())
        df['C'] = df['C'].str.lstrip('C').str.strip(' ')
        print(df['C'].isna().sum())
        df['D'] = df['D'].str.lstrip('D').str.strip(' ')
        print(df['D'].isna().sum())
        df['E'] = df['E'].str.lstrip('E').str.strip(' ')
        print(df['E'].isna().sum())
        df.to_csv(base_path+file,encoding='utf-8',index=False)
        
year='2023'
base_path = "./Data/"
enems = os.listdir(base_path)
for file in enems:
    if year in file:
        df = pd.read_csv('Data/'+file)
        for column in df:
            df[column] = df[column].str.replace(r' -(?=[A-Za-z])','-',regex=True)
            df[column] = df[column].str.replace(r'(?<=[A-Za-z])- ','-',regex=True)
            df[column] = df[column].str.replace(r' -(?=[\d])','-',regex=True)
            df[column] = df[column].str.replace(r'(?<=[\d])- ','-',regex=True)
        df.to_csv('Data/'+file,index=False)
        display(df.iloc[-3]['body'])

In [5]:
prova = 'ia2_caderno6_CAD_cinza_ledor_2013.csv'
df = pd.read_csv('data/ia2_caderno6_CAD_cinza_ledor_2013.csv ',encoding='utf-8')
year=2013
df['question'] = df['question'].apply(lambda question: question.upper())
df['year'] = year
df['question_number'] = df['question'].apply(lambda question: int(re.search("[\d]+",question).group()) if re.search("[\d]+",question) else np.nan)
df['test']='redação'
if 'D1_' in prova or 'ia1' in prova or 'DIA_1' in prova:
    if year < 2017:
        df.loc[df['question_number']<=45,'test'] = 'CH'
        df.loc[df['question_number']>45,'test'] = 'CN'
    else:
        df.loc[df['question_number']<=45,'test'] = 'LC'
        df.loc[df['question_number']>45,'test'] = 'CH'
else:
    if year < 2017:
        df.loc[df['question_number']<=135,'test'] = 'LC'
        df.loc[df['question_number']>135,'test'] = 'MT'
    else:
        df.loc[df['question_number']<=135,'test'] = 'CN'
        df.loc[df['question_number']>135,'test'] = 'MT'